In [ ]:
!pip install -q -U google-generativeai faiss-cpu openai

In [ ]:
import google.generativeai as genai
from openai import OpenAI
import numpy as np
import os
import numpy as np
import faiss

In [ ]:

API_KEY = "YOUR API KEY"
genai.configure(api_key=API_KEY) 

# Sample Data
sample_text = """
Artificial Intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems. 
These processes include learning (the acquisition of information and rules for using the information), reasoning (using rules to reach approximate or definite conclusions) and self-correction.
Particular applications of AI include expert systems, speech recognition and machine vision.

AI research has been highly successful in developing effective techniques for solving a wide range of problems, from game playing to medical diagnosis.
The history of AI began in antiquity, with myths, stories and rumors of artificial beings endowed with intelligence or consciousness by master craftsmen.
The seeds of modern AI were planted by classical philosophers who attempted to describe the process of human thinking as the mechanical manipulation of symbols.
This work culminated in the invention of the programmable digital computer in the 1940s, a machine based on the abstract essence of mathematical reasoning.
This device and the ideas behind it inspired a handful of scientists to begin seriously discussing the possibility of building an electronic brain.

The field of AI research was founded at a workshop held on the campus of Dartmouth College during the summer of 1956.
Those who attended would become the leaders of AI research for decades. Many of them predicted that a machine as intelligent as a human being would exist in no more than a generation and they were given millions of dollars to make this vision come true.
Eventually it became obvious that they had grossly underestimated the difficulty of the project.
In 1973, in response to the criticism of Sir James Lighthill and ongoing pressure from congress, the U.S. and British Governments stopped funding undirected research into artificial intelligence, and the difficult years that followed would later be known as an 'AI winter'.

Seven years later, a visionary initiative by the Japanese Government inspired governments and industry to provide AI with billions of dollars, but by the late 80s the investors became disillusioned and withdrew funding again.
This cycle of boom and bust, of 'AI springs' and 'AI winters', continues to haunt the field.
Nevertheless, AI has seen a resurgence in recent years due to advances in machine learning, big data, and computational power.
"""

In [ ]:

# Text chunking function
def chunk_text(text, chunk_size=300):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i:i + chunk_size])
    return chunks

# Split the sample text into chunks
chunks = chunk_text(sample_text)
print(f"Number of chunks: {len(chunks)}")
print("Sample chunk:", chunks[0])

In [ ]:
# Embeddings
embedding_model = "models/embedding-001"

def get_embedding(text):
    result = genai.embed_content(
        model=embedding_model,
        content=text,
        task_type="retrieval_document"
    )
    return result['embedding']

# Generate embeddings for each chunk
embeddings = [get_embedding(chunk) for chunk in chunks]
print(f"Embedding dimension: {len(embeddings[0])}")

In [ ]:
# Vector torage using FAISS
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)  # L2 distance index
index.add(np.array(embeddings).astype('float32'))
print(f"Number of vectors in index: {index.ntotal}")

In [ ]:
# Context retrieval function
def retrieve_context(query, k=3):
    query_emb = np.array([get_embedding(query)]).astype('float32')
    distances, indices = index.search(query_emb, k)
    retrieved_chunks = [chunks[idx] for idx in indices[0]]
    return retrieved_chunks

# Example query
query = "When was the field of AI founded?"
contexts = retrieve_context(query)
print("Retrieved contexts:")
for ctx in contexts:
    print(ctx[:100] + "...")

# generation
generative_model = genai.GenerativeModel('gemini-1.5-flash')  

def generate_answer(query, contexts):
    context_str = "\n\n".join(contexts)
    prompt = f"""
Answer the question based on the following context. If the context doesn't contain the answer, say "I don't know".

Context:
{context_str}

Question: {query}

Answer:
"""
    response = generative_model.generate_content(prompt)
    return response.text

# Example answer generation
answer = generate_answer(query, contexts)
print("Generated Answer:")
print(answer)


In [ ]:
def rag_query(query, k=3):
    contexts = retrieve_context(query, k)
    answer = generate_answer(query, contexts)
    return answer

# Test with another query
new_query = "What is AI winter?"
print("Query:", new_query)
print("Answer:", rag_query(new_query))
